### Домашнее задание №1 «Работа с Apache Spark»  
В файле movies.csv лежит база фильмов. Название фильма записано во втором столбце title.  
Разбейте названия фильмов на отдельные слова и посчитайте какое слово встречается чаще всего.  

Если у вас не получится со Спарком, то сделайте задание просто на питоне.  

In [1]:
from pyspark import SparkConf,SparkContext
import os

В терминале`SparkContext` создается автоматически как наберешь `pyspark`. В ноутбуке надо создавать вручную

In [2]:
conf = SparkConf()
conf.setAppName('df-project')  
sc = SparkContext(conf=conf)

22/11/27 21:07:11 WARN Utils: Your hostname, Victors-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.14 instead (on interface en0)
22/11/27 21:07:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/27 21:07:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/27 21:07:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
sc

<SparkContext master=local[*] appName=df-project>

In [4]:
data = sc.textFile('Downloads/movies.csv')

In [5]:
data.take(5)

['movieId,title,genres',
 '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance',
 '4,Waiting to Exhale (1995),Comedy|Drama|Romance']

Сперва надо убрать строку с заголовками

In [6]:
headers = data.first()
data = data.filter(lambda x: x != headers)

In [7]:
data.take(5)

['1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance',
 '4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 '5,Father of the Bride Part II (1995),Comedy']

Теперь норм.  
Делить строку будем по запятой, но на случай, если в названии есть запятая отбросим нулевой элемент и последний

In [8]:
data.map(lambda x: ' '.join(x.split(',')[:-1][1:]).strip('"')).take(5)

['Toy Story (1995)',
 'Jumanji (1995)',
 'Grumpier Old Men (1995)',
 'Waiting to Exhale (1995)',
 'Father of the Bride Part II (1995)']

Теперь разделим строку по пробелу и отбросим последний элемент с годом

In [9]:
data.map(
    lambda x: ' '.join(x.split(',')[:-1][1:]).strip('"')).flatMap(
    lambda x: x.split(' ')[:-1]).take(5)

['Toy', 'Story', 'Jumanji', 'Grumpier', 'Old']

Надо привести все к одному регистру и избавиться от встречающихся скобок по краям

In [10]:
data.map(
    lambda x: ' '.join(x.split(',')[:-1][1:]).strip('"')).flatMap(
    lambda x: x.split(' ')[:-1]).map(
    lambda x: x.lower().lstrip('(').rstrip(')')).take(10)

['toy',
 'story',
 'jumanji',
 'grumpier',
 'old',
 'men',
 'waiting',
 'to',
 'exhale',
 'father']

Все норм. надо добавить единицу для подсчетов

In [11]:
data.map(
    lambda x: ' '.join(
        x.split(',')[:-1][1:]).strip('"')).flatMap(
    lambda x: x.split(' ')[:-1]).map(
    lambda x: (x.lower().lstrip('(').rstrip(')'), 1)).take(10)

[('toy', 1),
 ('story', 1),
 ('jumanji', 1),
 ('grumpier', 1),
 ('old', 1),
 ('men', 1),
 ('waiting', 1),
 ('to', 1),
 ('exhale', 1),
 ('father', 1)]

Собственно подсчет

In [12]:
data.map(
    lambda x: ' '.join(x.split(',')[:-1][1:]).strip('"')).flatMap(
    lambda x: x.split(' ')[:-1]).map(
    lambda x: (x.lower().lstrip('(').rstrip(')'), 1)).reduceByKey(
    lambda a, b: a+b).take(15)

[('story', 73),
 ('jumanji', 1),
 ('grumpier', 1),
 ('waiting', 3),
 ('father', 16),
 ('of', 974),
 ('bride', 17),
 ('sabrina', 2),
 ('tom', 8),
 ('huck', 2),
 ('', 2281),
 ('dracula:', 1),
 ('loving', 2),
 ('nixon', 3),
 ('casino', 3)]

Отсортируем результат

In [13]:
data.map(
    lambda x: ' '.join(x.split(',')[:-1][1:]).strip('"')).flatMap(
    lambda x: x.split(' ')[:-1]).map(
    lambda x: (x.lower().lstrip('(').rstrip(')'), 1)).reduceByKey(
    lambda a, b: a+b).sortBy(
    lambda x: x[1], ascending=False).take(10)

[('the', 3109),
 ('', 2281),
 ('of', 974),
 ('a', 388),
 ('and', 353),
 ('in', 303),
 ('to', 212),
 ('man', 141),
 ('on', 119),
 ('a.k.a.', 115)]

In [14]:
path = f'{os.getcwd()}/dz_05.01/'  # Определяем путь до папки в которуюю поместим папки с результатами

In [15]:
sc.stop()